In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy
from sqlalchemy import create_engine, func, inspect, text

In [2]:
# Load in the csv
df = pd.read_csv("tornados.csv")
df.head(10)

,om,yr,mo,dy,date,time,tz,datetime_utc,st,stf,...,ns,sn,f1,f2,f3,f4,fc,region,month,num_tornados
0,192,1950,10,1,1950-10-01,21:00:00,America/Chicago,1950-10-02T03:00:00Z,OK,40,...,1,1,25,0,0,0,False,South,October,2499
1,193,1950,10,9,1950-10-09,02:15:00,America/Chicago,1950-10-09T08:15:00Z,NC,37,...,1,1,47,0,0,0,False,Southeast,October,1070
2,195,1950,11,20,1950-11-20,02:20:00,America/Chicago,1950-11-20T08:20:00Z,KY,21,...,1,1,177,0,0,0,False,South,November,904
3,196,1950,11,20,1950-11-20,04:00:00,America/Chicago,1950-11-20T10:00:00Z,KY,21,...,1,1,209,0,0,0,False,South,November,904
4,197,1950,11,20,1950-11-20,07:30:00,America/Chicago,1950-11-20T13:30:00Z,MS,28,...,1,1,101,0,0,0,False,South,November,2209
5,194,1950,11,4,1950-11-04,17:00:00,America/Chicago,1950-11-04T23:00:00Z,PA,42,...,1,1,71,11,0,0,False,Northeast,November,698
6,198,1950,12,2,1950-12-02,15:00:00,America/Chicago,1950-12-02T21:00:00Z,IL,17,...,1,1,119,117,0,0,False,Midwest,December,1407
7,199,1950,12,2,1950-12-02,16:00:00,America/Chicago,1950-12-02T22:00:00Z,IL,17,...,1,1,119,5,0,0,False,Midwest,December,1407
8,200,1950,12,2,1950-12-02,16:25:00,America/Chicago,1950-12-02T22:25:00Z,AR,5,...,1,1,65,0,0,0,False,South,December,1310
9,201,1950,12,2,1950-12-02,17:30:00,America/Chicago,1950-12-02T23:30:00Z,IL,17,...,1,1,157,0,0,0,False,Midwest,December,1407


In [5]:
# Check for null values, datatypes, all the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41516 entries, 0 to 41515
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   om            41516 non-null  int64  
 1   yr            41516 non-null  int64  
 2   mo            41516 non-null  int64  
 3   dy            41516 non-null  int64  
 4   date          41516 non-null  object 
 5   time          41516 non-null  object 
 6   tz            41516 non-null  object 
 7   datetime_utc  41516 non-null  object 
 8   st            41516 non-null  object 
 9   stf           41516 non-null  int64  
 10  mag           41516 non-null  float64
 11  inj           41516 non-null  int64  
 12  fat           41516 non-null  int64  
 13  loss          41516 non-null  float64
 14  slat          41516 non-null  float64
 15  slon          41516 non-null  float64
 16  elat          41516 non-null  float64
 17  elon          41516 non-null  float64
 18  len           41516 non-nu

In [3]:
# Save df to sqlite
engine = create_engine("sqlite:///tornados.sqlite")

df.to_sql("tornados", con=engine, if_exists="replace", index=False)

41516

In [6]:
# explore and understand the data
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within each table and its types
for table in tables:
    print(table)
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

tornados
om BIGINT
yr BIGINT
mo BIGINT
dy BIGINT
date TEXT
time TEXT
tz TEXT
datetime_utc TEXT
st TEXT
stf BIGINT
mag FLOAT
inj BIGINT
fat BIGINT
loss FLOAT
slat FLOAT
slon FLOAT
elat FLOAT
elon FLOAT
len FLOAT
wid BIGINT
ns BIGINT
sn BIGINT
f1 BIGINT
f2 BIGINT
f3 BIGINT
f4 BIGINT
fc BOOLEAN
region TEXT
month TEXT
num_tornados BIGINT



In [7]:
st = "All"

# allow the user to select ALL or a specific location
if st == "All":
    where_clause = "1=1"
else:
    where_clause = f"st = '{st}'"

query = f"""
        SELECT
            *
        FROM
            tornados
        WHERE
            {where_clause};
"""

print(query)


        SELECT
            *
        FROM
            tornados
        WHERE
            1=1;



In [8]:
# For map chart allow the user to select ALL or a specific state
region = "All"

if region == "All":
    where_clause = "1=1"
else:
    where_clause = f"region = '{region}'"

query = f"""
        SELECT
            *
        FROM
            tornados
        WHERE
            {where_clause};
    
"""

df_map = pd.read_sql(text(query),con=engine)
data_map = df_map.to_dict(orient="records")

df_map.head()

,om,yr,mo,dy,date,time,tz,datetime_utc,st,stf,...,ns,sn,f1,f2,f3,f4,fc,region,month,num_tornados
0,192,1950,10,1,1950-10-01,21:00:00,America/Chicago,1950-10-02T03:00:00Z,OK,40,...,1,1,25,0,0,0,0,South,October,2499
1,193,1950,10,9,1950-10-09,02:15:00,America/Chicago,1950-10-09T08:15:00Z,NC,37,...,1,1,47,0,0,0,0,Southeast,October,1070
2,195,1950,11,20,1950-11-20,02:20:00,America/Chicago,1950-11-20T08:20:00Z,KY,21,...,1,1,177,0,0,0,0,South,November,904
3,196,1950,11,20,1950-11-20,04:00:00,America/Chicago,1950-11-20T10:00:00Z,KY,21,...,1,1,209,0,0,0,0,South,November,904
4,197,1950,11,20,1950-11-20,07:30:00,America/Chicago,1950-11-20T13:30:00Z,MS,28,...,1,1,101,0,0,0,0,South,November,2209


In [9]:
# For bar chart allow the user to select ALL or a specific state

if region == "All":
    where_clause = "1=1"
else:
    where_clause = f"region = '{region}'"

query = f"""
    SELECT
        st as state,
        count(*) as num_tornados
    FROM
        tornados
    WHERE
        {where_clause}
    GROUP BY
        st
    ORDER BY
        num_tornados desc;
"""

df_bar = pd.read_sql(text(query), con=engine)
data_bar = df_bar.to_dict(orient="records")

df_bar.head()

,state,num_tornados
0,TX,4601
1,FL,2585
2,OK,2499
3,MS,2209
4,IA,1897


In [10]:
# For box plot chart allow the user to select ALL or a specific state
if st == "All":
    where_clause = "1=1"
else:
    where_clause = f"st = '{st}'"

query = f"""
    SELECT
        month,
        mag as magnitude
    FROM
        tornados
    WHERE
         {where_clause};
"""

df_box = pd.read_sql(text(query), con=engine)
data_box = df_box.to_dict(orient="records")

df_box.head()

,month,magnitude
0,October,1.0
1,October,3.0
2,November,2.0
3,November,1.0
4,November,1.0


In [11]:
# For sunburst chart allow the user to select ALL or a specific state
if region == "All":
     where_clause = "1=1"
else:
    where_clause = f"region = '{region}'"

query = f"""
    SELECT
        region as label,
        "" as parent,
        count(*) as num_tornados
    FROM
        tornados
    WHERE
        {where_clause}
    GROUP BY
        region
            
    UNION ALL 

    SELECT
        st as label,
        region as parent,
        count(*) as num_tornados
    FROM
        tornados
    WHERE
        {where_clause}
    GROUP BY
        st,
        region;
"""

df_sunburst = pd.read_sql(text(query), con=engine)
data_sunburst = df_sunburst.to_dict(orient="records")

df_sunburst.head()

,label,parent,num_tornados
0,Midwest,,14617
1,Northeast,,2040
2,South,,14565
3,Southeast,,8407
4,West,,1887
